# Programming Assignment 1: Spamlord

This assignment is your chance to become a Dark Lord of spam email! Yes, you too
can build regexes to spread evil throughout the galaxy.

Our goal in this assignment is to use regular expressions
to extract phone numbers and email addresses from documents found on
the web. If you really were a malicious actor, you could then use these
extracted addresses to bombard unsuspecting victims with spam!

Of course, we would never do anything nefarious like that in 124. Instead,
our goal will be to gain some experience with Python, Jupyter Notebooks, and
regular expressions!

## What we're trying to do

To make things a little more concrete, here are some examples to illustrate
exactly what your implementation should be able to do if it's working correctly!

### Extracting Phone Numbers

Your program should be able to process text that looks like the examples shown
on the left, and extract the phone number on the right (note that we want
the result in a standardized form).

```
# Various formats
Phone:  (650) 723-0293 =>  650-723-0293
Tel (+1): 650-723-0293 =>  650-723-0293

# HTML Markup
<a href="contact.html">TEL</a> +1&thinsp;650&thinsp;723&thinsp;0293 => 650-723-0293
```
__NOTE:__ You can assume we are only working with North American phone numbers,
so all numbers will be of the form: [3 digit area code]-[3 digits]-[4 digits].

### Extracting Email Addresses

Similarly to the phone numbers, we are also interested in processing text
containing (possibly obfuscated) email addresses and returning the corresponding
email addresses in a standard form.

```
# Ordinary email addresses
manning@cs.stanford.edu => manning@cs.stanford.edu

# Obfuscated email addresses
manning(at)cs.stanford.edu => manning@cs.stanford.edu
manning at csli dot stanford dot edu => manning@csli.stanford.edu

# Email addresses hidden using HTML/JavaScript
<script type="text/javascript">obfuscate('stanford.edu','manning')</script> => manning@stanford.edu
```

### Cases you won't have to worry about

Although you should aim to make your regexes as powerful and general-purpose as
you possibly can, there are some cases that are difficult or impossible to
handle with regexes and which we don't expect you to be able to deal with.

These include:

* Anything involving images or other non-text ways of displaying emails or phone
numbers
* Examples that require parsing names into parts, like:

```
"first name"@cs.stanford.edu
```

* Particularly clever/difficult examples that don't contain much or any
part of the actual email address. For example,

```
To send me email, try the simplest address that makes sense.
```

## Testing and Evaluation

Before you get started, here's some info about how your
implementation will be tested and evaluated.

### Some General Comments on Evaluation

In order to make your life easier on this (and future) homeworks, we will be
giving you some data to test your code on, which we call a
_development set_. Using a development set to test and evaluate your methods
is an extremely common approach in natural language processing and machine
learning. More generally, coming up with a robust set of test cases to
evaluate your work against is also an extremely important part of writing
good code.

In this case, our development set is a bunch of HTML documents (the personal
homepages of some Stanford CS professors) that we've scraped from the web and
downloaded for you. If you're not familiar with the details of HTML or its
syntax, that's fine. For the purposes of this assignment, all you need to know
is that the inputs are text files (with some formatting) that contain the
(possibly obfuscated) emails and phone numbers that we want to extract. You can
find all of these HTML documents in the ```data/dev``` folder.

In addition to the HTML documents, the ```data``` directory also contains
another file, ```data/devGOLD```. You can think of this file as the answer key
corresponding to the documents in ```data/dev```. It contains all the
correctly extracted phone numbers and emails from all the documents in
```data/dev```, in a particular format so your scripts (and our grading scripts)
can read them easily. The format is as follows:

### Format of Matches

Each line in the ```data/devGOLD``` file represents one extracted email address
or phone number in the form of a 3-tuple. Each tuple is represented as 3 strings
separated by tabs:

```
jurafsky	e	jurafsky@stanford.edu
```

The first string is the name of the file in ```data/dev``` that
the match came from.

The second string is an 'e' if the match was an email address, or a
'p' if it was a phone number.

The third string is the actual extracted email address or phone number itself,
in the following standard form:

```
  user@example.com
  650-555-1234
```

To sum up, the answers in the ```data/devGOLD``` file and the outputs
generated by your implementation should take the form of Python tuples
that look like:

```
  (filename, type, value)
```

__NOTE:__ You don't have to worry about case sensitivity in your values
(email addresses or phone numbers), since they will be normalized to lower case
before being compared with the answers.

### Scoring

Your grade will consist of two parts, totalling 24 points.

The first, worth 16 points, scores how well your implementation does on the
development set. For these examples you're given the correct answers,
so you should aim get 100% of them correct!

The second part of your grade, worth 8 points, will be based on how well your
regular expressions find emails and phone numbers in a different set of examples,
the test set. This test set is hidden and only the teaching staff knows what is
on it! Because you don't know exactly what trickery goes on in this test set,
you should be creative in looking at the web and thinking of different types
of ways of encoding emails and phone numbers, and not just rely on the methods
we've listed here, or are listed in the homework.

You're not expected to perform perfectly on the test set as you don't know
what is in it, or have the correct answers (just like in real life). As long as
you manage to achieve some reasonable performance (compared to a benchmark that
we provide), you'll get full points!

Here are the equations we use to calculate the scores for the two parts, where
`e` is the total number of errors (false negatives and false positives) for
each part:

__Dev:__

```
  if e < 10 then score(e) = 16 - e
  else if e >= 10 then score(e) = 6
```

__Test:__

```
  if e <= 10      then score(e) = 8
  else if 10 < e < 20 then score(e) = 13 - .5 * e
  if e >= 20      then score(e) = 3
```

__Note:__ This sort of two-stage evaluation (a known development set and a
hidden test set) is a very commonly used approach in machine learning!
Evaluating on a development set where we have the "right" answers lets us
measure our performance precisely and improve our approach, while a test set
that is hidden from us until later allows us to see how we perform
"out in the wild", on examples that we might not have been able to tailor
our methods to.

### Checking your Dev Set Performance

We provide a function, named `score()` for you to conveniently check your
performance on the development set as you work. It takes in a list of your
predicted matches (the output of the function you'll write) and a list of
correct/gold matches, read from the `data/devGOLD` file.

It compares the two and calculates how they overlap, printing out a
bunch of information in the following form:

```
  True Positives (4):
  set([('balaji', 'e', 'balaji@stanford.edu'),
       ('nass', 'e', 'nass@stanford.edu'),
       ('shoham', 'e', 'shoham@stanford.edu'),
       ('thm', 'e', 'pkrokel@stanford.edu')])
  False Positives (1):
  set([('psyoung', 'e', 'young@stanford.edu')])
  False Negatives (113):
  set([('ashishg', 'e', 'ashishg@stanford.edu'),
       ('ashishg', 'e', 'rozm@stanford.edu'),
       ('ashishg', 'p', '650-723-1614'),
       ('ashishg', 'p', '650-723-4173'),
       ('ashishg', 'p', '650-814-1478'),
  ...
```

The true positive section displays e-mails and phone numbers which are in
both your list of matches and the gold matches list. These are examples
that your regexes correctly found.

The false positive section displays matches which
your regular expressions extracted but which are not in the gold matches list.
These are incorrect and show where your method may have been too
broad/aggressive.

false negative section displays e-mails and phone numbers which your code did
not match, but which do exist in the html files. These are the matches your
code missed.

Your goal, then, is to reduce the number of false positives and false negatives
to zero. At the bottom of the output you can see the total counts of true
positives, false positives, and false negatives.

## Submitting your Solution

__IMPORTANT:__ Before you submit, make sure your code works in an environment
set up using Python 3.5.2 and the environment file and setup we provided
(and no other dependencies), as this is the
environment our autograder will be run on.

Submit your assignment
via [Gradescope (www.gradescope.com)](http://www.gradescope.com).
You can create a submission with the files below, and it will run the
remaining grading with the starter code.
We expect the following files in your final submission:

```
SpamLord.ipynb (do not alter the filename)
```

If your solution depends on other files, please put those files in a folder
named "deps/" (this folder should be on the same level as SpamLord.ipynb)
and upload a zip file (any name is fine) containing this folder and
SpamLord.ipynb to submit on Gradescope.
Gradescope will then automatically unzip the folder so that your
submission contains:

```
deps/
SpamLord.ipynb
```

## Frequently Asked Questions


__Q:__ Where should I be writing my Python code?

__A:__ The only place you should need to write code in this assignment is
the function `process_file(filename, data_directory)`.

```
  def process_file(filename, data_directory):
```

It's been marked for you with a "TODO" marker. This function takes
in a filename of a text file as a string and the path to the directory
 where the file is located, also as a string. It returns a list of tuples
representing e-mails or phone numbers found in that file. See the section above
about "Some General Comments on Evaluation" for a detailed description of the
output tuple format.

__Q:__ What version of Python should I use? How should I set up my dependencies?

__A:__ In this class, we will be using Python 3.5.2. Please see this document
 (Links to an external site.) for tips on how to set up Python 3.5.2.

__Q:__ What format should the phone numbers and e-mails have?
__A:__ The canonical forms we expect are:

```
  user@example.com
  650-555-1234
```

The case of the e-mails you find should not matter because the starter code
and the grading code will lowercase your matched e-mails before comparing
them against the gold set.

__Q:__ Where should I write/run my code?
__A:__ We strongly recommend

## Some Tips before you start:

* Although the phone number and email portions of the assignment are totally
independent and can be done in any order you'd like, we recommend starting with
phone numbers, as they are simpler and a good warm-up for the email regexes!
* If you need a refresher on Python or Jupyter notebooks (or you're new to
either of them), we strongly encourage taking a look at the intro to Python
notebook that we set up here: [link](). Referring back to it as you work might
help if you run into any issues with Python syntax or idioms.

## A Reminder about Jupyter Notebooks

Some of you may already be familiar with Python, but it's possible you haven't
yet done much work with Jupyter or IPython notebooks (or maybe everything is
equally new to you)!

If so, we __strongly__ encourage you to go back and read the Jupyter tips
section of the intro to Python notebook linked above.

If you don't have time to go look at it, here's a rundown of some common
issues/points of confusion when working with Jupyter notebooks:

* Every time you run a cell it changes the current state of your
notebook/kernel, just like if you ran the lines in the cell in an interactive
Python prompt.
* If you want to run a cell that depends on variables or functions
declared in previous cells, YOU MUST RUN THE PREVIOUS CELLS! Until you
run them, those variables/functions will not have been declared.
* If you re-run a cell that modifies or declares variables/functions, it will
overwrite the previous values/definitions (!).
* If things ever get really confusing and you'd like to start from scratch,
try restarting your Jupyter kernel.

## Getting Started!

Alright, let's extract some emails and phone numbers!

First, we'll import some packages that we'll need shortly:

In [45]:
# For examining the HTML files in our dev dataset
from IPython.display import display, HTML

# For opening files and manipulating file paths
from io import open
import os

# For regular expressions
import re

# Helper functions we'll use later
from util import process_dir, get_gold, score

Now, let's take a look at what our data actually looks like. This should always
be one of the first things you do whenever you're solving a problem that
requires working with data.

All of the data in our development set is found in files in the data/dev
directory. Each one is an HTML file scraped from a CS faculty webpage (although
they don't have a .html extension because we're going to be treating them
as just ordinary text files for our purposes).

Let's take a look at the contents of the one of the files to get a better
idea of what they actually look like.

In [48]:
# You can change this filename to look at other examples
example_filepath = 'data/dev/dabo'

# Display the HTML directly in our notebook
display(HTML(filename=example_filepath))

As expected, it looks just like a faculty webpage! It looks like there are a
few images that we weren't able to render because we didn't download them,
but that's fine because we're only interested in the text.

As is common for faculty pages, this one has an email address
and some phone numbers on it. Our goal is to write regular expressions
that we can use to automatically match and extract these from the webpages.

Feel free to change the filename to some of the other files in data/dev
(i.e. dabo to aiken, balaji, etc.) to take a look at some of the other
faculty pages. This is also one way to explore some of the different formats
that are used for email and phone numbers and start thinking of some ideas for
how to write your regular expressions.

Now we've seen what one of the files looks like as a webpage. However we're more
interested in working with and processing each file as raw text, as that's how
we'll be matching our regular expressions.

Let's try reading in the same file in text form, line-by-line to make sure
we are able to do that:

In [54]:
# Open and read from a file line-by-line
for line in open(example_filepath, 'r', encoding='ISO-8859-1'):
    if not line.isspace(): # Only print non-empty lines
        print(line)

<html>

<head>

<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">

<meta name="GENERATOR" content="Microsoft FrontPage 4.0">

<base href="http://crypto.stanford.edu/~dabo/index.html">

<title>Dan Boneh</title>

</head>

<body background="background.gif" text="#000000" link="#00006F" vlink="#003F00"

alink="#7F0000">

<div align="left">

<table border="0" cellspacing="0" cellpadding="15">

<tr> 

   <td></td>

<td>

<a href="pubs.html" 

	onMouseOver="pubs.src='icons/pubsON.gif';"

	onMouseOut="pubs.src='icons/pubs.gif';">

	<img border="0" src="icons/pubs.gif" alt="Publications" name="pubs"></a> <BR>

<a href="#courses"

	onMouseOver="courses.src='icons/coursesON.gif';"

	onMouseOut="courses.src='icons/courses.gif';">

	<img border="0" src="icons/courses.gif" alt="Courses" name="courses"></a> <BR>

<a href="http://crypto.stanford.edu" 

	onMouseOver="students.src='icons/studentsON.gif';"

	onMouseOut="students.src='icons/students.gif';">

	<img border="0" src="ic

Okay that looks reasonable! There's a bunch of somewhat cluttered HTML markup,
but it's all in text form and it looks like all of the text from the page
(including the emails and phone numbers!) are somewhere in here.

Now the interesting part: For each line,
instead of just reading and printing it,
let's try to find matches for a regular expression
that we provide, and extract those matches as emails.

In this case we'll use a super-simple pattern that just looks for 1 or more
alphanumeric characters or periods followed by an '@' followed by 1 or more
alphanumeric characters or periods, followed by '.edu'. This is just the usual
format of an email address.

Let's wrap that all together in a function and give it a whirl:

In [50]:
def example_process_file(filename: str, data_directory: str):
    """
    An example function we wrote to scan through a single file and return
    all the strings matching a simple hardcoded pattern.

    f will be an object satisfying the StringIO interface.
    You don't need to know the details of exactly what that entails, except
    that for your purposes f will always be a file object.
    """

    # This is the pattern we're trying to match
    # Note the use of two capture clauses, one before the '@' and one after the
    # '@'.
    simple_pattern = '([\w\.]+)@([\w\.]+.edu)'

    res = []
    for l in open(
            os.path.join(data_directory, filename),
            'r', encoding='ISO-8859-1'):
        # This function takes a regex pattern and a text string and returns
        # all matches in the string as a list
        # Each match in the list is a tuple of the capture groups in the
        # expression. So in this case, each element in matches will be a tuple
        # of form (stuff before '@', stuff after '@').
        matches = re.findall(simple_pattern, l)
        for m in matches:
            # Put the email back together
            email = '%s@%s' % m
            res.append((filename, 'e', email))

    return res

In [51]:
result = example_process_file('dabo', 'data/dev')
print(result)

[('dabo', 'e', 'dabo@cs.stanford.edu')]


Success! It looks like we got our first match! The output of our function in
this case is a list of matches, where each match is a tuple of
(file, 'e' or 'p' indicating email or phone number, extracted email or
phone number). The reason we want our output in this format is so that it works
with our automated evaluation later.

In this case we can see that we have just a single match from the
file data/dev/dabo (note that we only searched in this one file!) which is
an email and is the address 'dabo@cs.stanford.edu'.

Okay, so we've seen how we can process a single file. However, our dev
set consists of many such files. We need a function to loop over all of them,
process them, and return all the extracted addresses.

This function is provided for you in util.py! You will not need to modify it,
but we encourage you to take a look at how it is implemented.

Here's what happens when we run it:

In [53]:
all_results = process_dir('data/dev', example_process_file)

print(all_results)

[('manning', 'e', 'manning@cs.stanford.edu'), ('manning', 'e', 'manning@cs.stanford.edu'), ('manning', 'e', 'dbarros@cs.stanford.edu'), ('manning', 'e', 'dbarros@cs.stanford.edu'), ('manning', 'e', 'manning@cs.stanford.edu'), ('manning', 'e', 'manning@cs.stanford.edu'), ('zm', 'e', 'manna@cs.stanford.edu'), ('zm', 'e', 'manna@cs.stanford.edu'), ('nick', 'e', 'nick.parlante@cs.stanford.edu'), ('nick', 'e', 'nick.parlante@cs.stanford.edu'), ('cheriton', 'e', 'cheriton@cs.stanford.edu'), ('psyoung', 'e', 'patrick.young@stanford.edu'), ('psyoung', 'e', 'patrick.young@stanford.edu'), ('kunle', 'e', 'kunle@ogun.stanford.edu'), ('kunle', 'e', 'kunle@ogun.stanford.edu'), ('kunle', 'e', 'darlene@csl.stanford.edu'), ('kunle', 'e', 'darlene@csl.stanford.edu'), ('engler', 'e', 'engler@lcs.mit.edu'), ('eroberts', 'e', 'eroberts@cs.stanford.edu'), ('eroberts', 'e', 'eroberts@cs.stanford.edu'), ('rinard', 'e', 'rinard@lcs.mit.edu'), ('thm', 'e', 'pkrokel@Stanford.edu'), ('thm', 'e', 'pkrokel@Stanford

Looks like we got quite a few more matches, even with our very simple pattern.

You may have also noticed that our results have quite a few duplicates. If you
examine the corresponding files, you can see that this is happening because
the same email address (or phone number) appears more than once in the file.

Don't worry about this for now, we'll be careful to strip out duplicates later
when we're doing our evaluation.

Speaking of evaluation, before you go on to implement your own method let's
finish up by showing you the evaluation setup.

The evaluation process is straightforward: all that needs to be done is to load
the correct answers for the dev set from the provided file (data/devGOLD) and
compare them to the matches that were generated by your function.

We provide this helper function in util.py (you don't need to modify it, but
again we strongly encourage taking a peek at it to make sure that you understand
what it's doing).

Let's use it to read the gold (correct) matches from the provided file:

In [55]:
all_gold_matches = get_gold('data/devGOLD')
print(all_gold_matches)

[('ashishg', 'e', 'ashishg@stanford.edu'), ('ashishg', 'e', 'rozm@stanford.edu'), ('ashishg', 'p', '650-723-1614'), ('ashishg', 'p', '650-723-4173'), ('ashishg', 'p', '650-814-1478'), ('balaji', 'e', 'balaji@stanford.edu'), ('bgirod', 'p', '650-723-4539'), ('bgirod', 'p', '650-724-3648'), ('bgirod', 'p', '650-724-6354'), ('cheriton', 'e', 'cheriton@cs.stanford.edu'), ('cheriton', 'e', 'uma@cs.stanford.edu'), ('cheriton', 'p', '650-723-1131'), ('cheriton', 'p', '650-725-3726'), ('dabo', 'e', 'dabo@cs.stanford.edu'), ('dabo', 'p', '650-725-3897'), ('dabo', 'p', '650-725-4671'), ('dlwh', 'e', 'dlwh@stanford.edu'), ('engler', 'e', 'engler@lcs.mit.edu'), ('engler', 'e', 'engler@stanford.edu'), ('eroberts', 'e', 'eroberts@cs.stanford.edu'), ('eroberts', 'p', '650-723-3642'), ('eroberts', 'p', '650-723-6092'), ('fedkiw', 'e', 'fedkiw@cs.stanford.edu'), ('hager', 'e', 'hager@cs.jhu.edu'), ('hager', 'p', '410-516-5521'), ('hager', 'p', '410-516-5553'), ('hager', 'p', '410-516-8000'), ('hanrahan

As expected, these exactly match the output format that we showed earlier for
the file processing function. This will make it easy to compare our answers
to the gold answers.

We provide a helper function to do just that in util.py. Given that this
function shows you how your method will be scored, it's definitely worth
taking a look through.

Let's try evaluating our existing super-basic method using the above code:

In [57]:
guess_list = process_dir('data/dev', example_process_file)
gold_list = get_gold('data/devGOLD')
score(guess_list, gold_list)

Guesses (25): 
{('balaji', 'e', 'balaji@stanford.edu'),
 ('cheriton', 'e', 'cheriton@cs.stanford.edu'),
 ('dabo', 'e', 'dabo@cs.stanford.edu'),
 ('engler', 'e', 'engler@lcs.mit.edu'),
 ('eroberts', 'e', 'eroberts@cs.stanford.edu'),
 ('fedkiw', 'e', 'fedkiw@cs.stanford.edu'),
 ('hanrahan', 'e', 'hanrahan@cs.stanford.edu'),
 ('kosecka', 'e', 'kosecka@cs.gmu.edu'),
 ('kunle', 'e', 'darlene@csl.stanford.edu'),
 ('kunle', 'e', 'kunle@ogun.stanford.edu'),
 ('latombe', 'e', 'asandra@cs.stanford.edu'),
 ('latombe', 'e', 'latombe@cs.stanford.edu'),
 ('latombe', 'e', 'liliana@cs.stanford.edu'),
 ('manning', 'e', 'dbarros@cs.stanford.edu'),
 ('manning', 'e', 'manning@cs.stanford.edu'),
 ('nass', 'e', 'nass@stanford.edu'),
 ('nick', 'e', 'nick.parlante@cs.stanford.edu'),
 ('psyoung', 'e', 'patrick.young@stanford.edu'),
 ('rinard', 'e', 'rinard@lcs.mit.edu'),
 ('shoham', 'e', 'shoham@stanford.edu'),
 ('thm', 'e', 'pkrokel@stanford.edu'),
 ('widom', 'e', 'siroker@cs.stanford.edu'),
 ('widom', 'e', '

Looks reasonable! It appears that our basic method produced 25 matches,
while the gold set contains 117 matches.

There were 25 true positives (matches that we found that were in the gold set),
no false positives (matches that we found that were NOT in the gold set), and
92 false negatives (matches in the gold set that we did NOT find).

This seems like a pretty good start, but there are still 92 addresses that our
approach didn't manage to catch. Figuring out how to extract these addresses
without accidentally matching any non-address text is up to you!

You should write your implementation in the function below. Be sure to read the
documentation so that you know what sort of inputs/outputs will be expected.

In [64]:
# TODO: Implement your approach here!
def process_file(filename: str, data_directory: str):
    """
    This function takes in a filename, opens the corresponding file, and
    scans its contents against regex patterns. It returns a list of
    (filename, type, value) tuples where type is either an 'e' or a 'p'
    for e-mail or phone, and value is the formatted phone number or e-mail.
    The canonical formats are:
         (filename, 'p', '###-###-#####')
         (filename, 'e', 'someone@something')
    If the numbers you submit are formatted differently they will not
    match the gold answers.

    NOTE: DO NOT CHANGE THIS INTERFACE, as it will be called directly by
    the submit script
    """
    # TODO: Replace/modify this with your implementation
    res = []
    for l in open(
            os.path.join(data_directory, filename),
            'r', encoding='ISO-8859-1'):
        pass
    return res

As you work on your methods, you can use the code below to evaluate your
performance:

In [65]:
guess_list = process_dir('data/dev', process_file)
gold_list = get_gold('data/devGOLD')
score(guess_list, gold_list)

Guesses (0): 
set()
Gold (117): 
{('ashishg', 'e', 'ashishg@stanford.edu'),
 ('ashishg', 'e', 'rozm@stanford.edu'),
 ('ashishg', 'p', '650-723-1614'),
 ('ashishg', 'p', '650-723-4173'),
 ('ashishg', 'p', '650-814-1478'),
 ('balaji', 'e', 'balaji@stanford.edu'),
 ('bgirod', 'p', '650-723-4539'),
 ('bgirod', 'p', '650-724-3648'),
 ('bgirod', 'p', '650-724-6354'),
 ('cheriton', 'e', 'cheriton@cs.stanford.edu'),
 ('cheriton', 'e', 'uma@cs.stanford.edu'),
 ('cheriton', 'p', '650-723-1131'),
 ('cheriton', 'p', '650-725-3726'),
 ('dabo', 'e', 'dabo@cs.stanford.edu'),
 ('dabo', 'p', '650-725-3897'),
 ('dabo', 'p', '650-725-4671'),
 ('dlwh', 'e', 'dlwh@stanford.edu'),
 ('engler', 'e', 'engler@lcs.mit.edu'),
 ('engler', 'e', 'engler@stanford.edu'),
 ('eroberts', 'e', 'eroberts@cs.stanford.edu'),
 ('eroberts', 'p', '650-723-3642'),
 ('eroberts', 'p', '650-723-6092'),
 ('fedkiw', 'e', 'fedkiw@cs.stanford.edu'),
 ('hager', 'e', 'hager@cs.jhu.edu'),
 ('hager', 'p', '410-516-5521'),
 ('hager', 'p', '

__Best of luck!__

__Some reminders for submission:__
* Once you're satisfied with your code and have saved it in this
notebook, download it from Google Colab by going to File->Download .ipynb,
or by selecting it in the file explorer on the left and right-clicking to
download. If your solution requires any additional files to work, beyond the
notebook itself, be sure to place them in a `deps` folder at the same level
as `SpamLord.ipynb` and download that folder as well
* If you were working on your local machine, you don't need to do anything
* Zip up your `SpamLord.ipynb` (make sure the filename is correct!) and your
`deps` folder if you have one. You can do this by running
`zip submission.zip SpamLord.ipynb deps` from the command line, or do it
from your file explorer.
* Go to Gradescope (gradescope.com), find the PA1 SpamLord assignment and
upload the zip file.
* Wait for the autograder to run (it should only take a minute or so) and check
that your submission was graded successfully! If the autograder fails, or you
get an unexpected score it may be a sign that your zip file was incorrect.
* NOTE: When you view the autograder results, you'll only be able to see your
score on the development set (16 points).

